# Imports

In [1]:
import numpy as np

In [2]:
import os.path as osp
import numpy as np
from tqdm import tqdm
from utils import pickle_save, pickle_load
from pprint import pprint
from utils.data.delf import datum_io
from copy import deepcopy

In [3]:
import sacred
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from numpy import linalg as LA
from utils import pickle_load, pickle_save
#from utils.revisited import compute_metrics
from utils.data.delf import datum_io

In [4]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [5]:
feature_name = 'r50_gldv1'
set_name = 'tuto'
gnd_name = 'gnd_'+ set_name+'.pkl'

In [6]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [7]:
use_aqe = False
aqe_params = {'k': 2, 'alpha': 0.3}

save_nn_inds = True

In [8]:
with open(osp.join(data_dir,  set_name+'_query.txt')) as fid:
    query_lines   = fid.read().splitlines()

In [9]:
len(query_lines)

100

In [10]:
query_feats = []
for i in tqdm(range(len(query_lines))):
    name = osp.splitext(osp.basename(query_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, 'delg_' + feature_name, name + '.delg_global')
    query_feats.append(datum_io.ReadFromFile(path))

100%|██████████| 100/100 [00:00<00:00, 473.51it/s]


In [11]:
query_feats = np.stack(query_feats, axis=0)
query_feats = query_feats / LA.norm(query_feats, axis=-1)[:, None]

In [12]:
query_feats.shape

(100, 2048)

In [13]:
with open(osp.join(data_dir, set_name+'_selection.txt')) as fid:
    selection_lines = fid.read().splitlines()

In [14]:
np.stack(selection_lines, axis=0).shape

(4838,)

In [15]:
index_feats = []
for i in tqdm(range(len(selection_lines))):
    name = osp.splitext(osp.basename(selection_lines[i].split(';;')[0]))[0]
    path = osp.join(data_dir, 'delg_'+feature_name, name+'.delg_global')
    index_feats.append(datum_io.ReadFromFile(path))

100%|██████████| 4838/4838 [00:09<00:00, 492.02it/s]


In [16]:
selection_index_feats = np.zeros((query_feats.shape[0], 100, query_feats.shape[1]))
selection_index_feats.shape

(100, 100, 2048)

In [17]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))

In [18]:
selection_index_sizes = [len(gnd_data['simlist'][i]) for i in range(len(gnd_data['simlist']))]
np.sum(selection_index_sizes)

4838

In [19]:
size = 0
counter = 0
for i in range(selection_index_feats.shape[0]):
    for j in range(selection_index_feats.shape[1]):
        if j < selection_index_sizes[i]:
            selection_index_feats[i][j] = index_feats[counter]
            counter += 1

In [20]:
selection_index_feats.shape

(100, 100, 2048)

In [21]:
for i in range(selection_index_feats.shape[0]):
    selection_index_feats[i] = selection_index_feats[i]/LA.norm(selection_index_feats[i], axis=-1)[:,None]

/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [22]:
sims = []
for i in range(len(selection_index_feats)):
    index_feats = np.stack(selection_index_feats[i], axis=0)
    sims.append(np.matmul(query_feats[i], index_feats.T))

In [23]:
sims = np.stack(sims, axis=0)
sims.shape

(100, 100)

In [24]:
sims

array([[0.16223355, 0.16238563, 0.18141674, ...,        nan,        nan,
               nan],
       [0.17460734, 0.44644287, 0.3944457 , ...,        nan,        nan,
               nan],
       [0.25068201, 0.05434434, 0.44644287, ...,        nan,        nan,
               nan],
       ...,
       [0.0691188 , 0.14343122, 0.13620838, ...,        nan,        nan,
               nan],
       [0.21718944, 0.12931053, 0.14343122, ...,        nan,        nan,
               nan],
       [0.14343122, 0.0806773 , 0.21771399, ...,        nan,        nan,
               nan]])

In [25]:
if use_aqe:
    alpha = aqe_params['alpha']
    nn_inds = np.argsort(-sims, -1)
    query_aug = deepcopy(query_feats)
    for i in range(len(query_feats)):
        new_q = [query_feats[i]]
        for j in range(aqe_params['k']):
            nn_id = nn_inds[i, j]
            weight = sims[i, nn_id] ** aqe_params['alpha']
            new_q.append(weight * index_feats[nn_id])
        new_q = np.stack(new_q, 0)
        new_q = np.mean(new_q, axis=0)
        query_aug[i] = new_q/LA.norm(new_q, axis=-1)
    sims = np.matmul(query_aug, index_feats.T)

In [26]:
selection_index_feats[0].shape

(100, 2048)

In [27]:
nn_inds = np.argsort(-sims, -1)
nn_dists = deepcopy(sims)
for i in range(query_feats.shape[0]):
    index_feats = selection_index_feats[i]
    for j in range(index_feats.shape[0]):
        nn_dists[i, j] = sims[i, nn_inds[i, j]]

In [28]:
nn_inds.shape

(100, 100)

In [29]:
selection_index_sizes[10]

28

In [30]:
max(nn_inds[10][:28])

27

In [31]:
if save_nn_inds:
    output_path = osp.join(data_dir, set_name + '_nn_inds_%s.pkl' % feature_name)
    pickle_save(output_path, nn_inds)

In [32]:
def compute_ap(ranks, nres):
    """
    Computes average precision for given ranked indexes.
    
    It assumes that `ranks` contains the ranks for all expected positive
    index images to be retrieved. If `positive_ranks` is empty, returns
    `average_precision` = 0.
    Note that average precision computation here does NOT use the finite sum
    method (see
    https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Average_precision)
    which is common in information retrieval literature. Instead, the method
    implemented here integrates over the precision-recall curve by averaging two
    adjacent precision points, then multiplying by the recall step. This is the
    convention for the Revisited Oxford/Paris datasets.
    
    Arguments
    ---------
    ranks : zerro-based ranks of positive images
    nres  : number of positive images
    
    Returns
    -------
    ap    : average precision
    
    """

    # number of images ranked by the system
    nimgranks = len(ranks)

    # accumulate trapezoids in PR-plot
    ap = 0

    recall_step = 1. / nres

    for j in np.arange(nimgranks):
        rank = ranks[j]

        if rank == 0:
            precision_0 = 1.
        else:
            precision_0 = float(j) / rank

        precision_1 = float(j + 1) / (rank + 1)

        ap += (precision_0 + precision_1) * recall_step / 2.

    return ap

In [33]:
def compute_map(ranks, gnd, kappas=[]):
    """
    Computes the mAP for a given set of returned results.

         Usage: 
           map = compute_map (ranks, gnd) 
                 computes mean average precsion (map) only
        
           map, aps, pr, prs = compute_map (ranks, gnd, kappas) 
                 computes mean average precision (map), average precision (aps) for each query
                 computes mean precision at kappas (pr), precision at kappas (prs) for each query
        
         Notes:
         1) ranks starts from 0, ranks.shape = db_size X #queries
         2) The junk results (e.g., the query itself) should be declared in the gnd stuct array
         3) If there are no positive images for some query, that query is excluded from the evaluation
    """

    map = 0.
    nq = len(gnd) # number of queries
    aps = np.zeros(nq)
    pr = np.zeros(len(kappas))
    prs = np.zeros((nq, len(kappas)))
    nempty = 0

    for i in np.arange(nq):
        qgnd = np.array(gnd[i]['ok'])
        qgndj = np.array(gnd[i]['junk'])

        # no positive groundtruth images, skip from the average
        if qgnd.shape[0] == 0:
            aps[i] = 0
            prs[i, :] = 0
            continue

        # only negative images retrieved in the IR step
        if qgnd.shape[0] > 100:
            aps[i] = 0.
            prs[i, :] = 0.
            continue
        
        # sorted positions of positive and junk images (0 based)
        pos  = np.arange(ranks.shape[0])[np.isin(ranks[:,i], qgnd)]
        junk = np.arange(ranks.shape[0])[np.isin(ranks[:,i], qgndj)]

        k = 0
        ij = 0
        if len(junk):
            # decrease positions of positives based on the number of
            # junk images appearing before them
            ip = 0
            while (ip < len(pos)):
                while (ij < len(junk) and pos[ip] > junk[ij]):
                    k += 1
                    ij += 1
                pos[ip] = pos[ip] - k
                ip += 1

        # compute ap
        ap = compute_ap(pos, len(qgnd))
        map = map + ap
        aps[i] = ap

        # compute precision @ k
        pos += 1 # get it to 1-based
        for j in np.arange(len(kappas)):
            if len(pos) == 0:
                max_pos = kappas[j]
            else: max_pos = max(pos)
            kq = min(max_pos, kappas[j]); 
            prs[i, j] = (pos <= kq).sum() / kq
        pr = pr + prs[i, :]

    map = map / nq # (nq - nempty)
    pr = pr /nq # (nq - nempty)

    return map, aps, pr, prs, nempty

In [34]:
gnd_data = pickle_load(osp.join(data_dir, gnd_name))

In [35]:
nn_inds.T[:,1], gnd_data['gnd'][1]['hard']

(array([ 7, 19,  1, 20,  2,  5, 17, 16, 15,  8, 18, 14,  4,  9,  0,  6, 12,
        10,  3, 13, 11, 77, 76, 75, 74, 73, 72, 71, 70, 67, 68, 66, 65, 64,
        63, 62, 78, 69, 79, 89, 81, 97, 96, 95, 94, 93, 92, 91, 80, 90, 88,
        87, 86, 85, 84, 83, 82, 61, 60, 49, 58, 36, 35, 34, 33, 32, 31, 30,
        37, 29, 27, 26, 25, 24, 23, 22, 21, 28, 38, 39, 40, 57, 56, 55, 54,
        53, 52, 51, 50, 98, 48, 47, 46, 45, 44, 43, 42, 41, 59, 99]),
 ['512px-Jack_Ruby-1.jpg'])

In [36]:
np.where(nn_inds.T[:,0]==10)

(array([16]),)

In [37]:
pos  = np.arange(nn_inds.T.shape[0])[np.isin(nn_inds.T[:,0], gnd_data['gnd'][0]['hard'])]
junk = np.arange(nn_inds.T.shape[0])[np.isin(nn_inds.T[:,0], gnd_data['gnd'][0]['junk'])]
pos, junk

/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/numpy/lib/arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(array([], dtype=int64), array([], dtype=int64))

In [38]:
def compute_metrics(dataset, ranks, gnd, kappas=[1, 5, 10]):
    print(ranks.shape)
    
    # old evaluation protocol
    if dataset.startswith('classic'):
        map, aps, _, _ = compute_map(ranks, gnd)
        out = {'map': np.around(map*100, decimals=3)}
        print('>> {}: mAP {:.2f}'.format(dataset, out['map']))

    # new evaluation protocol
    elif dataset.startswith('viquae'):
        
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['r_easy'],gnd[i]['r_hard']])
            g['junk'] = np.concatenate([gnd[i]['r_neg']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH, nempty = compute_map(ranks, gnd_t, kappas)


        out = {
            'H_map': np.around(mapH*100, decimals=2),
            'H_mp':  np.around(mprH*100, decimals=2),
        }

        print('>> {}: mAP H: {}'.format(dataset, out['H_map']))
        print('>> {}: mP@k{} H: {}'.format(dataset, kappas, out['H_mp']))

    return out, mapH, apsH, mprH, prsH, nempty

In [39]:
np.array(gnd_data['gnd'][10]['junk']).shape[0]

21

In [40]:
np.arange(nn_inds.T.shape[0])[np.in1d(nn_inds.T[:,i], gnd_data['gnd'][10]['hard'])], gnd_data['gnd'][10]['hard']

(array([], dtype=int64), [])

In [41]:
out, mapH, apsH, mprH, prsH, nempty = compute_metrics('viquae', nn_inds.T, gnd_data['gnd'][:5], kappas=[1,5,6,10,100])

(100, 100)
>> viquae: mAP H: 97.8
>> viquae: mP@k[1, 5, 6, 10, 100] H: [100. 100. 100. 100. 100.]


In [42]:
prsH

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [43]:
from ranx import Qrels, Run, evaluate

/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/ranx/qrels_run_common.py:7: UserWarning: Sorting disabled. Assumes that you provided sorted doc_ids!
  warnings.warn("Sorting disabled. Assumes that you provided sorted doc_ids!")


In [45]:
gnd_data['gnd'][i]['r_easy']

[]

In [46]:
qrels_dict = {}
run_dict = {}
for i in range(nn_inds.shape[0]):
    q_str = 'q_' + str(i)
    
    qrels_dict[q_str] = dict([('d_' + str(i) + '_' + str(key), 1) for key in np.concatenate([gnd_data['gnd'][i]['r_easy'],gnd_data['gnd'][i]['r_hard']])])
    run_dict[q_str] = dict([('d_' + str(i) + '_' + str(key), 1) for key in nn_inds.T[:,i]])    

In [47]:
from ranx import Qrels, Run

qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [48]:
from ranx import evaluate

evaluate(qrels, run, ["map", "map@1", "map@5", "map@10",
                      "mrr", "mrr@1", "mrr@5", "mrr@10",
                      "precision", "precision@1", "precision@5", "precision@10"])
# ValueError: Metric  not supported. Supported metrics are `hits`, `hit_rate`,
# `precision`, `recall`, `f1`, `r-precision`, `mrr`, `map`, `ndcg`, and `ndcg_burges`.

{'map': 0.08992761775481943,
 'map@1': 0.020639898989898993,
 'map@5': 0.0386410101010101,
 'map@10': 0.05397412377745711,
 'mrr': 0.13697181964573268,
 'mrr@1': 0.12,
 'mrr@5': 0.13449999999999998,
 'mrr@10': 0.13616666666666666,
 'precision': 0.021799999999999996,
 'precision@1': 0.12,
 'precision@5': 0.07400000000000001,
 'precision@10': 0.068}

In [ ]:
def compute_metrics(dataset, ranks, gnd, kappas=[1, 5, 10]):
   
    from ranx import Qrels, Run, evaluate
    qrels_dict = {}
    run_dict = {}
    for i in range(ranks.T.shape[0]):
        q_str = 'q_' + str(i)

        qrels_dict[q_str] = dict([('d_' + str(i) + '_' + str(key), 1) for key in np.concatenate([gnd[i]['r_easy'],gnd[i]['r_hard']])])
        run_dict[q_str] = dict([('d_' + str(i) + '_' + str(key), 1) for key in ranks[:,i]])    
    
    qrels = Qrels(qrels_dict)
    run = Run(run_dict)

    out = evaluate(qrels, run, ["map", "map@"+str(kappas[0]), "map@"+str(kappas[1]), "map@"+str(kappas[2]),
                      "mrr", "mrr@"+str(kappas[0]), "mrr@"+str(kappas[1]), "mrr@"+str(kappas[2]),
                      "precision", "precision@"+str(kappas[0]), "precision@"+str(kappas[1]), "precision@"+str(kappas[2])])

    return out

In [ ]:
for i in range(len(gnd_data['gnd'])):
    intersection = set(gnd_data['gnd'][i]['r_easy']).intersection(set(gnd_data['gnd'][i]['r_hard']))
    if len(list(intersection)) > 0:
        print(i)

In [ ]:
i = 1
gnd_data['gnd'][i]['r_easy'], gnd_data['gnd'][i]['r_hard']

In [ ]:
compute_metrics('viquae', nn_inds.T, gnd_data['gnd'], kappas=[1, 5, 10])